# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
## Load Cities.csv as DataFrame
cities_df = pd.read_csv('Output/Cities.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Lat', 'Lng']].values.tolist()
type(locations[0][0])

float

In [5]:
max_humidity = cities_df['Humidity'].max()
max_humidity

100

In [252]:
fig = gmaps.figure(center=(15,15), zoom_level=2.1)

locations = cities_df[['Lat', 'Lng']].values.tolist()
weights = cities_df['Humidity'].values.tolist()
max_humidity = cities_df['Humidity'].max()

heat_layer = gmaps.heatmap_layer(
    locations=locations,
    weights=weights,
    dissipating=False,
    max_intensity=max_humidity,
    point_radius = 3
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
len(cities_df)

538

In [9]:
ideal_conditions = (cities_df['Max Temp'] > 70) & \
                   (cities_df['Max Temp'] < 80) & \
                   (cities_df['Wind Speed'] < 10) & \
                   (cities_df['Cloudiness'] == 0)

In [37]:
sum(ideal_conditions)

8

In [105]:
ideal_cities_df = cities_df[['City', 'Country', 'Lat', 'Lng']].loc[ideal_conditions]
ideal_cities_df.reset_index(drop=True, inplace=True)
ideal_cities_df.dropna(axis=0, inplace=True)

In [106]:
ideal_cities_df

,City,Country,Lat,Lng
0,Baghdad,IQ,33.3406,44.4009
1,Oussouye,SN,12.4850,-16.5469
2,Najrān,SA,17.4924,44.1277
3,Karratha,AU,-20.7377,116.8463
4,Abu Kamal,SY,34.4506,40.9171
5,Mount Isa,AU,-20.7333,139.5000
6,Tura,IN,25.5198,90.2201
7,Maxixe,MZ,-23.8597,35.3472


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [213]:
## GOOGLE MAPS API

## Nearby Search URL
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Nearby Search Parameters
params = {
    'key': g_key,
    'radius': 5000,
    'type': "lodging",
    'language': "en"
}

In [263]:
hotel_name = []
hotel_lat = []
hotel_lng = []

for index, row in ideal_cities_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(nearby_url, params=params).json()
    first_result = response['results'][0]

    hotel_name.append(first_result['name'])
    hotel_lat.append(first_result['geometry']['location']['lat'])
    hotel_lng.append(first_result['geometry']['location']['lng'])
    

In [264]:
hotel_df = ideal_cities_df[['City', 'Country']]
hotel_df.insert(0, 'Hotel Name', hotel_name)
hotel_df.insert(len(hotel_df.columns), 'Lat', hotel_lat)
hotel_df.insert(len(hotel_df.columns), 'Lng', hotel_lng)

hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Baghdad Hotel,Baghdad,IQ,33.318668,44.415058
1,Auberge du Routard,Oussouye,SN,12.488839,-16.550006
2,محمد هادي ال فطيح,Najrān,SA,17.489915,44.128800
3,ibis Styles Karratha,Karratha,AU,-20.732225,116.846744
4,محطة التلفاز,Abu Kamal,SY,34.449376,40.903100
5,ibis Styles Mt Isa Verona,Mount Isa,AU,-20.726025,139.492599
6,Hotel RIKMAN Continental,Tura,IN,25.512665,90.217633
7,Oceano,Maxixe,MZ,-23.861426,35.346690


In [254]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [255]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))